In [ ]:
%load_ext autoreload
%autoreload 2

# ZFEL example

In [ ]:
import os, sys
sys.path.insert(0, os.path.abspath('/Users/nneveu/github/zfel'))
from zfel import sase1d

import zfel.plot

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## Input 

In [ ]:
sase_input = dict(
    npart   = 2048,                       # n-macro-particles per bucket 
    s_steps = 200,                      # n-sample points along bunch length
    z_steps = 200,                      # n-sample points along undulator
    energy  = 1000.e6,                # electron energy [eV]
    eSpread = 1e-3,                       # relative rms energy spread [1]
    emitN   = 0.4e-6,                    # normalized transverse emittance [m-rad]
    currentMax = 1000,                   # peak current [Ampere]
    beta = 12,                          # mean beta [meter]
    unduPeriod = 0.026,                 # undulator period [meter]
    unduK = 2.442 ,          # undulator parameter, K [1], array could taper. 
    unduL = 50,                         # length of undulator [meter]
    radWavelength=None,                 # Will calculate based on resonance condition for unduK[0]
    random_seed=31,                     # for reproducibility
    particle_position=None, #np.genfromtxt('./Inputs/particle_position.csv', delimiter=',') # or None,
    hist_rule='square-root',             # 'square-root' or 'sturges' or 'rice-rule' or 'self-design', number \
                                       #  of intervals to generate the histogram of eta value in a bucket
    iopt='sase',
    P0 = 1e0                            # small seed input power [W]
)

In [ ]:
lamda_r

## Intermediate parameters 

These are used internally

In [ ]:
params = sase1d.params_calc(**sase_input)
params.keys()

## Run

In [ ]:
%%time
output = sase1d.sase(sase_input)

In [ ]:
output.keys()

## Plot output

In [ ]:
plt.ylabel('Power (GW)')
plt.xlabel('z (m)')
plt.plot(output['z'], output['power_z']/1e9)

In [ ]:
# Power evolution. z vs slice
plt.imshow( (output['Er']**2 + output['Ei']**2).T, origin='lower')

In [ ]:
plt.semilogy(output['z'], 1e-9*output['power_z'])
plt.xlabel("Undulator distance, m")
plt.ylabel("Power (log scale), GW")
plt.xlim([0, 20])
plt.twinx()
plt.plot(output['z'], 1e-9*output['power_z'], 'r')
plt.ylabel("Power (lin scale), GW")
plt.ylim([0, 0.5*10E0])
plt.show()

In [ ]:
zfel.plot.plot_log_power_z(output)

In [ ]:
zfel.plot.plot_power_s(output)

In [ ]:
# plot spectrum
freq = output['freq']
spectrum = output['spectrum']
plt.plot(freq,spectrum[1:,-1])
plt.xlabel('energy (eV)')
plt.ylabel('spectrum (W/?)')